In [17]:
import pandas as pd
import datetime as dt

In [67]:
data = pd.read_csv("./retailtransactiondata/Retail_Data_Transactions.csv",parse_dates=["trans_date"])
BUDGET_AMOUNT = 2000

In [68]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125000 entries, 0 to 124999
Data columns (total 3 columns):
customer_id    125000 non-null object
trans_date     125000 non-null datetime64[ns]
tran_amount    125000 non-null int64
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 2.9+ MB


In [69]:
data.head(10)

,customer_id,trans_date,tran_amount
0,CS5295,2013-02-11,35
1,CS4768,2015-03-15,39
2,CS2122,2013-02-26,52
3,CS1217,2011-11-16,99
4,CS1850,2013-11-20,78
5,CS5539,2014-03-26,81
6,CS2724,2012-02-06,93
7,CS5902,2015-01-30,89
8,CS6040,2013-01-08,76
9,CS3802,2013-08-20,75


In [70]:
customer_data = data.groupby("customer_id")
last_transaction = customer_data["trans_date"].max()
churn_data = pd.DataFrame(last_transaction)
churn_data.head(5)

,trans_date
customer_id,
CS1112,2015-01-14
CS1113,2015-02-09
CS1114,2015-02-12
CS1115,2015-03-05
CS1116,2014-08-25


In [71]:
churn_data['trans_date'].value_counts()

2015-03-15    101
2015-03-06     98
2015-03-16     94
2015-03-09     93
2015-03-03     86
             ... 
2014-04-14      1
2014-03-23      1
2014-07-10      1
2014-04-22      1
2014-02-15      1
Name: trans_date, Length: 414, dtype: int64

In [72]:
cutoff_day = dt.datetime(2015, 1, 15)
churn_data["churned"] = churn_data["trans_date"].apply(lambda date:1 if date < cutoff_day else 0)
churn_data[churn_data["churned"] ==1]

,trans_date,churned
customer_id,,
CS1112,2015-01-14,1
CS1116,2014-08-25,1
CS1117,2014-07-02,1
CS1123,2014-11-27,1
CS1124,2015-01-02,1
...,...,...
CS8994,2014-12-17,1
CS8996,2014-12-09,1
CS8997,2014-06-28,1


In [73]:
churn_data["amount_spent"] = customer_data.sum()
churn_data["nr_of_transactions"] = customer_data.size()
#Date has served its purpose so getting rid of it
churn_data.drop("trans_date", axis="columns", inplace=True)
churn_data

,churned,amount_spent,nr_of_transactions
customer_id,,,
CS1112,1,1012,15
CS1113,0,1490,20
CS1114,0,1432,19
CS1115,0,1659,22
CS1116,1,857,13
...,...,...,...
CS8996,1,582,13
CS8997,1,543,14
CS8998,1,624,13


In [74]:
churn_data[
    ["nr_of_transactions", "amount_spent"]
].describe().loc[["min", "max"]]

,nr_of_transactions,amount_spent
min,4.0,149.0
max,39.0,2933.0


In [75]:
churn_data["scaled_tran"] = (churn_data["nr_of_transactions"] - churn_data["nr_of_transactions"].min()) \
                             / (churn_data["nr_of_transactions"].max() - churn_data["nr_of_transactions"].min())

churn_data["scaled_amount"] = (churn_data["amount_spent"] - churn_data["amount_spent"].min()) \
                             / (churn_data["amount_spent"].max() - churn_data["amount_spent"].min())
churn_data["score"] = 100 * (.5 * churn_data["scaled_tran"]                +.5 * churn_data["scaled_amount"])
churn_data.sort_values("score", inplace=True, ascending=False)
churn_data 

,churned,amount_spent,nr_of_transactions,scaled_tran,scaled_amount,score
customer_id,,,,,,
CS4424,0,2933,39,1.000000,1.000000,100.000000
CS4320,1,2647,38,0.971429,0.897270,93.434934
CS3799,1,2513,36,0.914286,0.849138,88.171182
CS5109,0,2506,35,0.885714,0.846624,86.616892
CS3805,1,2453,35,0.885714,0.827586,85.665025
...,...,...,...,...,...,...
CS7716,1,221,4,0.000000,0.025862,1.293103
CS7224,1,191,4,0.000000,0.015086,0.754310
CS8504,0,190,4,0.000000,0.014727,0.736351


In [92]:
coupon_value = int(data["tran_amount"].mean() *.32)
print("Coupon value: ",coupon_value)
number_coupons = BUDGET_AMOUNT/coupon_value
print("Number of Coupons: ",number_coupons)

Coupon value:  20
Number of Coupons:  100.0


In [93]:
top_100_val_customers = churn_data.loc[churn_data["churned"] == 1].head(100)

top_100_val_customers.to_csv("valuable_customers.txt")